In [1]:
import pandas as pd
import sqlite3
import random

conn = sqlite3.connect('aroma.db')
df = pd.read_sql_query('SELECT * FROM items', conn)
conn.close()

In [12]:
import re

def calculate_string(s):
    s = str(s).replace('духи', '')
    s = re.sub(r'(\d+),(\d+)', r'\1.\2', s)

    numbers = re.findall(r'\d+\.?\d*', s)
    operators = re.findall(r'[+\-*xхх]', s)

    numbers = [float(num) for num in numbers]

    i = 0
    while i < len(operators):
        op = operators[i]
        if op == '*' or op == 'x' or op == 'х' or op == 'х':
            if op == '*' or op == 'x' or op == 'х' or op == 'х':
                result = numbers[i] * numbers[i+1]
            else:
                result = numbers[i] / numbers[i+1]
            numbers[i:i+2] = [result]
            del operators[i]
        else:
            i += 1

    result = numbers[0]
    for i, op in enumerate(operators):
        next_num = numbers[i+1] if i+1 < len(numbers) else 0
        if op == '+' or op == 'и':
            result += next_num
        elif op == '-':
            result -= next_num

    return result

In [12]:
df['volume'] = df['volume'].apply(calculate_string)

In [13]:
import random

def get_dimensions(volume):
    if 3 <= volume <= 14:
        weight = random.randint(42, 60)
        length = random.randint(44, 64)
        width = random.randint(44, 44)
        height = random.randint(55, 69)
    elif 15 <= volume <= 45:
        weight = random.randint(60, 150)
        length = random.randint(60, 100)
        width = random.randint(50, 100)
        height = random.randint(60, 140)
    elif 45 <= volume <= 99:
        weight = random.randint(150, 250)
        length = random.randint(120, 150)
        width = random.randint(50, 100)
        height = random.randint(80, 170)
    elif 99 <= volume:
        weight = volume + 100
        length = 150
        width = 100
        height = 170
    
    return weight, length, width, height

df[['weight', 'length', 'width', 'height']] = df['volume'].apply(lambda x: pd.Series(get_dimensions(x)))

In [14]:
search_str = ["ZARKOPERFUME", "Acqua di Parma", "Abercrombie & Fitch", "Aerin", "Giorgio Armani", "Armani", "Amouage", "Balmain", "Benetton", "Boadicea The Victorious", "BYBOZO", "Byredo", "Carolina Herrera", "Calvin Klein", "CK", "Creed", "Diptyque", "Donna Karan", "Christian Dior", "Ermenegildo Zegna", "Etat Libre D`Orange", "Etat", "Evody", "Ex Nihilo", "Franck Olivier", "Giardino Benessere", "Guess", "Gucci", "Hermes", "Hermetica", "Histoires de Parfums", "Jacques Zolty", "Jeanne Arthes", "John Varvatos", "Juicy Couture", "Couture", "Juliette has a Gun", "Lanvin", "Lacoste", "Lancome", "Les Contes", "Michael Kors", "Moresque", "Olfactive Studio", "Paco Rabanne", "Penhaligon's", "Philipp Plein", "Ralph Lauren", "Stella McCartney", "Tiziana Terenzi", "Tom Ford", "Tommy Hilfiger", "Tous", "V Canto", "Vilhelm Parfumerie", "Zarkoperfume", "Versace", "Givenchy", "Memo", "Hermes", "Jo Malone", 'аква ди парма', 'аберкромби & фитч', 'аерин', 'Амуаж', 'Бальмен', 'Бальмэйн', 'Бенеттон', 'Бодичеа Викториос', 'Байбозо', 'Биредо', 'Каролина Херрера', 'Кельвин Кляйн', 'Крид', 'Диптик', 'Донна Каран', 'Диор', 'Ерменегильдо Зегна', 'Этат', 'Эводи', 'Экс Нихило', 'Фрэнк Оливер', 'Жардино Бенессере', 'Гесс', 'Гуччи', 'Гермес', 'Герметика', 'Хисторес де парфюмс', 'Жакье Золти', 'Жан Артес', 'Джон Варватос', 'Джуси Кутюр', 'Майкл Корс', 'Мореск', 'Ольфактив Студио', 'Пако Рабанне', 'Пенхалигонс', 'Филипп Пляйн', 'Ральф Лорен', 'Стелла МакКартни', 'Тизмана Терензи', 'Том Форд', 'Томми хилфигер', 'Той', 'В Канто', 'Вильгельм Парфюмерие', 'Заркоперфюм', 'Версаче', 'Дживанши', 'Мемо', 'Гермес', 'Джо Малон']
for item in search_str:
    mask = df['brand'].astype(str).apply(lambda y: bool(re.search(re.escape(item.lower()), y.lower(), re.IGNORECASE)))
    df.loc[mask, 'brand'] = "НЕТ БРЕНДА"

delete_str = ["Antonio Banderas", "Hugo Boss", "Carolina Herrera", "Chanel", "Clarins", "Dior", "Dolce & Gabbana", "Escentric Molecules", "Armani", "Givenchy", "Lancome", "Lanvin", "Versace", 'Антонио Бандерас', 'Хуго Босс', 'Каролина Херрера', 'Шанел', 'Кларинс', 'Диор', 'Дольче & Габанна', 'Ескентрик Молекулес', 'Армани', 'Дживанши', 'Ланкоме', 'Ланвин', 'Версаче', 'Хьюго Босс', 'Клэринс', 'Клеринс', 'Дольче Габана', 'Дольче Габбана', 'Живанши', 'Ланком']
for item in delete_str:
    mask = df.apply(lambda row: row.astype(str).str.contains(r'\b' + re.escape(item) + r'\b', case=False, regex=True).any(), axis=1)
    df = df[~mask]

replace_str = ''
for item in search_str:
    mask = df['item_name'].astype(str).apply(lambda y: bool(re.search(r'\b' + re.escape(item.lower()) + r'\b' + ', ', y.lower(), re.IGNORECASE)))
    df.loc[mask, 'item_name'] = df.loc[mask, 'item_name'].apply(lambda x: re.sub(r'\b' + re.escape(item) + r'\b' + ', ', replace_str, x, flags=re.IGNORECASE) if isinstance(x, str) else x)

for item in search_str:
    mask = df['annotation'].astype(str).apply(lambda y: bool(re.search(r'\b' + re.escape(item.lower()) + r'\b', y.lower(), re.IGNORECASE)))
    df.loc[mask, 'annotation'] = df.loc[mask, 'annotation'].apply(lambda x: re.sub(r'\b' + re.escape(item) + r'\b', replace_str, x, flags=re.IGNORECASE) if isinstance(x, str) else x)

In [ ]:
countries = ["Австралия", "Австрия", "Азербайджан", "Аргентина", "Бахрейн", "Бельгия", "Великобритания", "Германия", "Греция", "Дания", "Египет", "Индия", "Ирландия", "Исландия", "Испания", "Италия", "Канада", "Катар", "Китай", "Кувейт", "Латвия", "Ливан", "Монако", "Нидерланды", "ОАЭ", "Оман", "Польша", "Португалия", "Россия", "США", "Саудовская Аравия", "Сирия", "Словакия", "Словения", "Турция", "Украина", "Финляндия", "Франция", "Чехия", "Швейцария", "Швеция", "Япония"]

df.loc[~df['country'].isin(countries), 'country'] = ''

In [ ]:
df['image_url'] = df['image_url'].str.replace('thumbnail_images', 'original_images')

In [ ]:
df = df[~df['item_name'].str.contains('тестер')]

In [ ]:
df['okpd_code'] = 'ОКПД - 20.42.11 - Духи и туалетная вода'

In [15]:
grouped = df.groupby('commerce_type')

for commerce_type, group_df in grouped:
    filename = f'{commerce_type}.xlsx'
    group_df.to_excel(filename, index=False)
